In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
fake_news = pd.read_csv("C:/Users/Dell/Downloads/Fake.csv")
true_news = pd.read_csv("C:/Users/Dell/Downloads/True.csv")
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [3]:
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
fake_news["Fake"] = 1
true_news["Fake"] = 0

In [5]:
fake_news.shape , true_news.shape

((23481, 5), (21417, 5))

In [6]:
fake_news_manual_testing = fake_news.tail(10)
for i in range(23470,23480,1):
    fake_news.drop([i], axis = 0 , inplace = True)
true_news_manual_testing = fake_news.tail(10)
for i in range(21406,21416,1):
    true_news.drop([i], axis = 0 , inplace = True)

In [7]:
fake_news.shape , true_news.shape

((23471, 5), (21407, 5))

In [8]:
#assigning 0 to fake news and 1 to the true news of the manual data
fake_news_manual_testing["Fake"] = 0
true_news_manual_testing["Fake"] = 1

C:\Users\Dell\AppData\Local\Temp\ipykernel_8244\584998597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_news_manual_testing["Fake"] = 0
C:\Users\Dell\AppData\Local\Temp\ipykernel_8244\584998597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_news_manual_testing["Fake"] = 1


In [12]:
# merging the remaining data
data_merge = pd.concat([fake_news,true_news], axis = 0)
data_merge.shape

(44878, 5)

In [13]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'Fake'], dtype='object')

In [14]:
#drop the not required fields
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [15]:
data.head()

,text,Fake
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1


In [16]:
#checking null data
data.isnull().sum()

text    0
Fake    0
dtype: int64

In [17]:
#Suffling data
data = data.sample(frac = 1)

In [18]:
data

,text,Fake
1100,WASHINGTON (Reuters) - U.S. President Donald T...,0
20245,PORT-AU-PRINCE (Reuters) - French President Em...,0
13445,GENEVA (Reuters) - The United Nations called o...,0
11045,WASHINGTON (Reuters) - U.S. Vice President Joe...,0
5362,Infamous Italian dictator Benito Mussolini ant...,1
...,...,...
17908,WELLINGTON (Reuters) - The leader of the small...,0
2356,Penguin Random House announced on Tuesday nigh...,1
6104,WASHINGTON (Reuters) - President Donald Trump ...,0
15734,BEIRUT (Reuters) - Syrian government forces wi...,0


In [19]:
# Preprocessing text
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", "", text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+','', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [17]:
data['text'] = data['text'].apply(wordopt)

In [20]:
# Dependent and independent variables
x = data['text']
y = data['Fake']

In [21]:
x_train, x_test, y_train , y_test = train_test_split( x, y, test_size = 0.25)

In [22]:
## Assigning vectors to the tokens
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [23]:
# Model formation

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [24]:
#predicting the test data
pred_lr = LR.predict(xv_test)

In [25]:
LR.score(xv_test, y_test)

0.9854723707664884

In [26]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5428
           1       0.99      0.98      0.99      5792

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [27]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [28]:
pred_dt = DT.predict(xv_test)

In [29]:
DT.score(xv_test, y_test)

0.9949197860962566

In [30]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5428
           1       0.99      1.00      1.00      5792

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [32]:
from sklearn.ensemble import GradientBoostingClassifier

GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [33]:
pred_gb = GB.predict(xv_test)

In [34]:
GB.score(xv_test, y_test)

0.9950089126559715

In [35]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5428
           1       1.00      0.99      1.00      5792

    accuracy                           1.00     11220
   macro avg       0.99      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



In [36]:
# Random Forrest
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [37]:
pred_rf = RF.predict(xv_test)

In [38]:
RF.score(xv_test, y_test)

0.9899286987522281

In [39]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5428
           1       0.99      0.99      0.99      5792

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



In [59]:
def output_label(n):
    if n == 1:
        return "Fake News"
    elif n == 0:
        return " True News"

def manual_testing(news):
    testing_news = {"text": [news]}
    news_def_test = pd.DataFrame(testing_news)
    news_def_test["text"] = news_def_test["text"].apply(wordopt)
    news_x_test = news_def_test["text"]
    news_xv_test = vectorization.transform(news_x_test)
    pred_lr = LR.predict(news_xv_test)
    pred_DT = DT.predict(news_xv_test)
    pred_GB = GB.predict(news_xv_test)
    pred_RF = RF.predict(news_xv_test)
    return print("\n\n LR Prediction: {} \n DT Prediction: {} \n GB Prediction: {} \n RF Prediction : {}".format(output_label(pred_lr[0]), output_label(pred_DT[0]), output_label(pred_GB[0]), output_label(pred_RF[0])))

In [60]:
news = str(input())
manual_testing(news)

 JAKARTA (Reuters) - Indonesia will buy 11 Sukhoi fighter jets worth $1.14 billion from Russia in exchange for cash and Indonesian commodities, two cabinet ministers said on Tuesday. The Southeast Asian country has pledged to ship up to $570 million worth of commodities in addition to cash to pay for the Suhkoi SU-35 fighter jets, which are expected to be delivered in stages starting in two years. Indonesian Trade Minister Enggartiasto Lukita said in a joint statement with Defence Minister Ryamizard Ryacudu that details of the type and volume of commodities were  still being negotiated . Previously he had said the exports could include palm oil, tea, and coffee. The deal is expected to be finalised soon between Indonesian state trading company PT Perusahaan Perdangangan Indonesia and Russian state conglomerate Rostec. Russia is currently facing a new round of U.S.-imposed trade sanctions. Meanwhile, Southeast Asia s largest economy is trying to promote its palm oil products amid threat



 LR Prediction: Fake News 
 DT Prediction: Fake News 
 GB Prediction: Fake News 
 RF Prediction : Fake News
